<a href="https://colab.research.google.com/github/sridevi-1234/MLops_git/blob/master/langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LangChain Memory**

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

## **Building memory into a system**
The two core design decisions in any memory system are:
- How state is stored
- How state is queried

## **What's covered?**
- ConversationBufferMemory
- End-to-end Example: Conversational AI Bot
- Saving and Loading a Chat History
- ConversationBufferWindowMemory

## **ConversationBufferMemory**

Let's take a look at how to use ConversationBufferMemory in chains. ConversationBufferMemory is an extremely simple form of memory that just keeps a list of chat messages in a buffer and passes those into the prompt template.

This memory type can be connected to a conversation. It allows for storing messages and then extracts the messages in a variable.

## **End-to-end Example: Conversational AI Bot**

<img src="images/memory.png">

### **Steps:**
1. Import Chat Model and Configure the API Key
2. Create Chat Template
3. Initialize the Memory
4. Create a Output Parser
5. Build a Chain
6. Invoke the chain with human_input and chat_history
7. Saving to memory
8. Run Step 6 and 7 in a loop

# Step 1 : Import Chat Model and Configure the API Key

In [ ]:

from langchain_openai import ChatOpenAI
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [ ]:
# ********************************* Example start ********************************

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.load_memory_variables({})

C:\Users\KUMAR SUNDRAM\AppData\Local\Temp\ipykernel_26008\1868966598.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': ''}

In [ ]:
memory.buffer

''

In [ ]:
memory = ConversationBufferMemory(memory_key = "chat_history")
memory.chat_memory.add_user_message("Hi!")
memory.chat_memory.add_ai_message("Hi, what's up? How can I help you?")
memory.load_memory_variables({})

{'chat_history': "Human: Hi!\nAI: Hi, what's up? How can I help you?"}

In [ ]:
memory.buffer

"Human: Hi!\nAI: Hi, what's up? How can I help you?"

In [ ]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)
memory.chat_memory.add_user_message("Hi!")
memory.chat_memory.add_ai_message("Hi, what's up? How can I help you?")
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi, what's up? How can I help you?", additional_kwargs={}, response_metadata={})]}

In [ ]:
# ********************************* Example End ********************************

# Step 2 - Create Chat Template

In [ ]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human."),
    # creating a chat_history placeholder
    MessagesPlaceholder(variable_name="chat_history"),
    # Human Prompt
    HumanMessagePromptTemplate.from_template("{human_input}"),])


# Step 3 - Initialize the Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)

# Step 4 - Create a Output Parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# Step 5 - Build a Chain (Another way)

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define a function to load the message from memory
def get_messages_from_memory(human_input):
    return memory.load_memory_variables(human_input)['chat_history']

# Define a chain
chain = RunnablePassthrough.assign(chat_history = RunnableLambda(get_messages_from_memory)) |  chat_prompt_template | chat_model | output_parser

# Step 6 - Invoke the chain with human_input and chat_history

In [ ]:
query= {"human_input":"Hi, How are you?"}
response = chain.invoke(query)
response

"Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?"

# Step 7 - Saving to memory

In [ ]:
memory.save_context(query, {'output': response})
memory.buffer

[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={})]

# Step 8 - Run Step 6 and 7 in a loop

In [ ]:
while True:
    query= {"human_input": input('Enter your input here:  ')}
    print(f"*User: {query['human_input']}")
    if query['human_input'] in ['bye', 'quit', 'exit']:
        break
    response = chain.invoke(query)
    print(f"*AI : {response}")
    memory.save_context(query, {'output': response})

Enter your input here:   Hi, how are you?


*User: Hi, how are you?
*AI : Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?


Enter your input here:   ok, fine. no problem. how is your health now?


*User: ok, fine. no problem. how is your health now?
*AI : As a computer program, I don't have health in the way humans do. But thank you for asking! How can I assist you today?


Enter your input here:   I love you


...

*User: can you please explain 1st Distance matrix in detail and step by step?
*AI : Certainly! Let's dive into the details of the Euclidean distance, which is one of the most commonly used distance metrics in K-Nearest Neighbors (KNN) algorithm.

The Euclidean distance between two points A(x1, y1) and B(x2, y2) in a 2-dimensional space is calculated using the formula:
\[ d_{AB} = \sqrt{{(x2 - x1)^2 + (y2 - y1)^2}} \]

Here's a step-by-step explanation of how to calculate the Euclidean distance between two points:

1. **Identify the coordinates of the two points:** Let's say we have two points A(3, 4) and B(6, 8).

2. **Calculate the differences in the x and y coordinates:** The differences are:
   - \( x2 - x1 = 6 - 3 = 3 \)
   - \( y2 - y1 = 8 - 4 = 4 \)

3. **Square the differences:** Square each difference:
   - \( (x2 - x1)^2 = 3^2 = 9 \)
   - \( (y2 - y1)^2 = 4^2 = 16 \)

4. **Sum the squared differences:** Add the squared differences together:
   - \( (x2 - x1)^2 + (y2 - y1)^2 = 

Enter your input here:   Can you please suggest what skillset required to become Data Analytics and Data Scientist?


*User: Can you please suggest what skillset required to become Data Analytics and Data Scientist?
*AI : To become a successful Data Analyst or Data Scientist, you will need a combination of technical skills, domain knowledge, and soft skills. Here are some key skills required for each role:

**Data Analyst:**
1. **Statistical Analysis:** Understanding of statistical concepts and techniques for data analysis.
2. **Data Manipulation:** Proficiency in tools like SQL, Excel, and data visualization tools like Tableau or Power BI.
3. **Programming:** Knowledge of programming languages like Python or R for data cleaning and analysis.
4. **Data Visualization:** Ability to create meaningful and insightful visualizations to communicate findings effectively.
5. **Domain Knowledge:** Understanding of the industry or field in which you are working to derive actionable insights.
6. **Critical Thinking:** Analytical mindset to interpret data and draw meaningful conclusions.

**Data Scientist:**
1. **

Enter your input here:   bye


*User: bye


In [ ]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi, how are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='ok, fine. no problem. how is your health now?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="As a computer program, I don't have health in the way humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I love you', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Thank you! I'm here to assist you

In [ ]:
memory.buffer

[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ok, fine. no problem. how is your health now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="As a computer program, I don't have health in the way humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love you', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Thank you! I'm here to assist you with any questions or t

# Saving a Chat History

In [ ]:
import pickle

In [ ]:
import os
os.getcwd()

'C:\\Users\\KUMAR SUNDRAM\\OneDrive\\Desktop\\GEN AI\\Introduction to LangChain\\6. Memory'

In [ ]:
chat_history = pickle.dumps(memory)
with open("conversation_memory.pkl","wb") as f:
    f.write(chat_history)

# Loading a Chat History

In [ ]:
chat_history_loaded = pickle.load(open("conversation_memory.pkl", "rb"))
print(chat_history_loaded)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='ok, fine. no problem. how is your health now?', additional_kwargs={}, response_metadata={}), AIMessage(content="As a computer program, I don't have health in the way humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='I love you', additional_kwargs={}, response_metadata={}), AIMessage(content="Thank you! I'm h

## **SQLChatMessageHistory**

`ChatMessageHistory` allows us to store separate conversation histories per user or session which is often done by the real-time chatbots. `session_id` is used to distinguish between separate conversations.

In order to use it, we can use a `get_session_history` function which take `session_id` and returns a message history object.

There is a support of many `Memory` components under `langchain_community.chat_message_histories`, like:
1. AstraDBChatMessageHistory
2. DynamoDBChatMessageHistory
3. CassandraChatMessageHistory
4. ElasticsearchChatMessageHistory
5. KafkaChatMessageHistory
6. MongoDBChatMessageHistory
7. RedisChatMessageHistory
8. PostgresChatMessageHistory
9. SQLChatMessageHistory

**[Click Here](https://python.langchain.com/v0.2/docs/integrations/memory/)** to read more.

### **Usage**

To use the storage you need to provide only 2 things:

1. Session Id - a unique identifier of the session, like user name, email, chat id etc.
2. Connection string
    - For SQL (SQLAlchemy) - A string that specifies the database connection. It will be passed to SQLAlchemy create_engine function.
    - For SQLite - A string that specifies the database connection. For SQLite, that string is slqlite:/// followed by the name of the database file. If that file doesn't exist, it will be created.

In [ ]:
from langchain_openai import ChatOpenAI
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [ ]:
#!pip install langchain_community

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(session_id=session_id, connection='sqlite:///chats_data/sqlite.db')
    return chat_message_history

In [ ]:
# Creat a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages([('system', "You are a helpful AI assistant."),
                                                  MessagesPlaceholder(variable_name = "history"),
                                                  ("human","{human_input}")])

In [ ]:
# Chain
chain = chat_template | chat_model

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# Define a chain
conversation_chain = RunnableWithMessageHistory(chain, get_session_message_history_from_db,
                                                input_messages_key="human_input",
                                                history_messages_key = "history")

In [ ]:
# This is where we configure the session id

user_id = "sundram"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input": "My name is Sundram. Can you tell me the Capital of Gujarat?"}
response = conversation_chain.invoke(input_prompt, config=config)
response.content

'Hello Sundram! The capital of Gujarat is Gandhinagar.'

In [ ]:
user_id = "sundram"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input": "My name is Sundram. What are the biggest 3 states in India?"}
response = conversation_chain.invoke(input_prompt, config=config)
print(response.content)

I apologize for the mistake. The three largest states in India by area are:

1. Rajasthan
2. Madhya Pradesh
3. Maharashtra

Thank you for your patience, Sundram. Let me know if you have any more questions.


In [ ]:
user_id = "sundram"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input": "My name is Sundram. What are the top 5 states of India on the basis of population?"}
response = conversation_chain.invoke(input_prompt, config=config)
print(response.content)

I apologize for the oversight. Here are the top 5 most populous states in India:

1. Uttar Pradesh
2. Maharashtra
3. Bihar
4. West Bengal
5. Madhya Pradesh

Thank you for bringing that to my attention, Sundram. Let me know if you need any more information.


In [ ]:
user_id = "Urmil"
config = {"configurable":{"session_id":user_id}}
input_prompt = {"human_input": "My name is Urmil. What is the biggest state in India?"}
response = conversation_chain.invoke(input_prompt, config=config)
response.content

'The biggest state in India by area is Rajasthan.'

In [ ]:
def chat_bot(session_id, prompt):
    config={"configurable":{"session_id":user_id}}
    input_prompt = {"human_input": prompt}
    response = conversation_chain.invoke(input_prompt, config=config)
    return response.content

In [ ]:
user_id = "Urmil"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

'Yes, your name is Urmil. How can I assist you today, Urmil?'

In [ ]:
user_id = "Vijay"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

"I'm sorry, but I don't have the ability to remember personal information about users unless it has been shared with me in the course of our conversation. How can I assist you today?"

In [ ]:
user_id = "sundram"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

'Yes, your name is Sundram. How can I assist you today?'